<a href="https://colab.research.google.com/github/tukamilano/combinatory_logic/blob/main/combinator_logic_cython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext Cython

In [2]:
!pip install tqdm

In [3]:
%%cython
from itertools import product
from cpython cimport bool

special_token_list = {"B": [3, ["A", 1, "A", 2, 3]], "T": [2, ["A", 2, 1]], "M": [1, ["A", 1, 1]]}
variable_token_list = []
token_list = list(special_token_list.keys()) + variable_token_list

cpdef repeat_reduce(str term, str reduction_order="left", int max_repeat=10, int max_length=100):
    cdef int i = 0
    cdef list reduction_history = [term]
    cdef str next_term
    cdef bool reduced

    while i < max_repeat:
        next_term, reduced = calculator_reduce(term, reduction_order)
        i += 1
        if term != next_term:
            reduction_history.append(next_term)
        if reduced == True:
            return reduction_history
        if len(term) > max_length:
            return reduction_history
        term = next_term

    return reduction_history

cpdef calculator_reduce(str term, str reduction_order):
    cdef bool reduced = True
    cdef list scan
    cdef int i
    cdef str special_token
    cdef int depth
    cdef int pointer
    cdef list separator
    cdef int num_placeholder
    cdef str new_term
    cdef repl
    cdef list next_term_list = []

    if (reduction_order == "left") or (reduction_order == "all"):
        scan = list(range(len(term)))
    else:
        scan = list(reversed(range(len(term))))

    for i in scan:
        for special_token in special_token_list:
            depth = special_token_list[special_token][0]
            if (depth <= i) and (term[i-depth:i+1] == "A" * depth + special_token):
                pointer = i+1
                separator = [pointer]
                for _ in range(depth):
                    num_placeholder = 1
                    while num_placeholder != 0:
                        if term[pointer] == "A":
                            num_placeholder += 1
                        else:
                            num_placeholder -= 1
                        pointer += 1
                    separator.append(pointer)

                new_term = term[:i-depth]
                for repl in special_token_list[special_token][1]:
                    if isinstance(repl, str):
                        new_term += repl
                    else:
                        new_term += term[(separator[repl-1]):(separator[repl])]
                new_term += term[pointer:]
                reduced = False

                if (reduction_order == "left") or (reduction_order == "right"):
                    return new_term, reduced
                else:
                    next_term_list.append(new_term)

    if (reduction_order == "left") or (reduction_order == "right"):
        return term, reduced
    else:
        return next_term_list

cpdef _generate_fixed_structure(int n, str sequence='', int a_count=0, int l_count=0, list results=None):
    if results is None:
        results = []

    if len(sequence) == 2 * n:
        results.append(sequence + "V")
        return results

    if a_count < n:
        _generate_fixed_structure(n, sequence + 'A', a_count + 1, l_count, results)
    if l_count < a_count:
        _generate_fixed_structure(n, sequence + 'V', a_count, l_count + 1, results)

    return results

cpdef _replace_variables(list all_structures, list token_list):
    cdef list new_sequences = []
    cdef str sequence
    cdef int v_count
    cdef replacement
    cdef str new_sequence
    cdef var

    for sequence in all_structures:
        v_count = sequence.count("V")
        for replacement in product(token_list, repeat=v_count):
            new_sequence = sequence
            for var in replacement:
                new_sequence = new_sequence.replace("V", var, 1)
            new_sequences.append(new_sequence)

    return new_sequences

cpdef variable_generate_sequences(list token_list, int length):
    cdef list all_structures = []
    cdef int n
    cdef list structure
    cdef list all_sequences

    for n in range(0, length):
        structure = _generate_fixed_structure(n)
        all_structures.extend(structure)

    all_sequences = _replace_variables(all_structures, token_list)
    print(f"Total number of sequences after variable replacement: {len(all_sequences)}")
    return all_sequences

cpdef all_repeat_reduce(str term, int max_size=100, int max_repeat=10):
    cdef int i = 0
    cdef list all_term_list = [term]
    cdef list next_term_list

    while i < len(all_term_list) < max_repeat:
        next_term_list = calculator_reduce(all_term_list[i], reduction_order="all")
        if next_term_list == []:
            break
        for next_term in next_term_list:
            if next_term not in all_term_list:
                all_term_list.append(next_term)
        i += 1

    return all_term_list

cpdef bool definitionally_equal(str term1, str term2, int max_size=100):
    if set(all_repeat_reduce(term1, max_size)) & set(all_repeat_reduce(term2, max_size)) != set():
        return True
    else:
        return False

In [4]:
from tqdm import tqdm

def identification(candidate_list):
  clean_candidate_list = []
  for candidate in tqdm(candidate_list):
    ident = False
    for clean_candidate in clean_candidate_list:
      if definitionally_equal("AAA" + candidate + "XYZ", "AAA" + clean_candidate + "XYZ"):
        ident = True
        break
    if not ident:
      clean_candidate_list.append(candidate)
  print(f"Total number of clean_candidate: {len(clean_candidate_list)}")
  return clean_candidate_list


In [5]:
from itertools import permutations

candidate_list = variable_generate_sequences(token_list, 7)
#clean_candidate_list = identification(candidate_list)

Total number of sequences after variable replacement: 323175


In [6]:
for candidate in candidate_list:
    if definitionally_equal("AAA" + candidate + "XYZ", "AAXZAYZ"):
        print(candidate)
        break